# Preprocessing

In [1]:
# import libraries
import numpy as np
from sklearn.model_selection import cross_val_score

In [2]:
# clone github repo
!git clone https://github.com/ch-zheng/cchs-prediction.git
%cd cchs-prediction/

fatal: destination path 'cchs-prediction' already exists and is not an empty directory.
/content/cchs-prediction


In [3]:
# load data
samples = np.load('data/samples.npy')
labels = np.load('data/labels.npy')

In [4]:
# initialize dictionary
avg_accuracy = {}
avg_accuracy["Multilayer Perceptron"] = 0.900 # mlp.py

# **Decision Tree**

In [5]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

In [6]:
scores = cross_val_score(model, samples, labels, cv=100)
avg_accuracy["Decision Tree"] = sum(scores) / len(scores)

# **Logistic Regression**

In [7]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)

In [8]:
scores = cross_val_score(model, samples, labels, cv=100, n_jobs=-1)
avg_accuracy["Logistic Regression"] = sum(scores) / len(scores)

# **LASSO Regression**

In [9]:
#tktk

# **Polynomial Regression**

In [10]:
#tktk

# **Ridge Regression**

In [11]:
from sklearn.linear_model import RidgeClassifier
model = RidgeClassifier()

In [12]:
scores = cross_val_score(model, samples, labels, cv=100)
avg_accuracy["Ridge Regression"] = sum(scores) / len(scores)

# **K-nearest neighbors**

In [13]:
from sklearn import neighbors
model = neighbors.KNeighborsClassifier(n_jobs=-1)

In [14]:
scores = cross_val_score(model, samples, labels, cv=100)
avg_accuracy["K-Nearest Neighbors"] = sum(scores) / len(scores)

# **Support Vector Machine**

In [15]:
from sklearn import svm
model = svm.SVC()

In [16]:
scores = cross_val_score(model, samples, labels, cv=100, n_jobs=-1)
avg_accuracy["SVM"] = sum(scores) / len(scores)

# **Naive Bayes**

In [17]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [18]:
scores = cross_val_score(model, samples, labels, cv=100)
avg_accuracy["Naive Bayes"] = sum(scores) / len(scores)

# Display Metrics

In [31]:
# display avg accuracy
for i in sorted(avg_accuracy, key=avg_accuracy.get, reverse=True):
  print("%-30s%-20s" % (i, "{:.2f}".format(avg_accuracy[i]*100)))

Multilayer Perceptron         90.00               
Ridge Regression              81.94               
Logistic Regression           78.05               
K-Nearest Neighbors           71.52               
Decision Tree                 69.76               
SVM                           68.68               
Naive Bayes                   53.77               
